In [12]:
import numpy as np

%matplotlib notebook
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from utils import *

DATA_PATH = '../../datav2/esp/'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
df = load_data(DATA_PATH,'train',nclasses=5).loc[:,['review_content','review_rate']]
df.head(5)

Dataset cargado para 5 clases (muy malo=1, malo=2, medio=3, bueno=4 muy bueno=5)
Num samples per category:
1    92477
2    92449
3    92487
4    92501
5    92454
Name: review_rate, dtype: int64


,review_content,review_rate
0,"Esta de muy baja calidad, no fue lo esperado.",2
1,"Reconozco que fue muy económico, pero su durab...",2
2,Muy bien muy bien muy bien muy bien muy bien m...,5
3,No me fue útil para mí. Diseño muy bueno.,2
4,No fue lo estipulado. Solo eso voy aclarar.,1


In [11]:
import pickle

wd = 0.001
with open('results_hyper_search/checkpoint_0.0_0.0001_{}.pkl'.format(wd), 'rb') as f:
    data = pickle.load(f)

y = data[0.0][0.0001][wd]['y']
y_pred = data[0.0][0.0001][wd]['y_pred']

print('accuracy en dev: {:.2f}%'.format((y==y_pred).mean()*100))

accuracy en dev: 57.45%


In [21]:
from hyperparams_search import BertClassifier, BERT_PATH

clf = BertClassifier(model_path=BERT_PATH,n_iters=10,
                batch_size=16,learning_rate=1e-4,weight_decay=0.0,
                n_classes=5,device='cuda:1')

Some weights of the model checkpoint at /home/lestien/Documents/Trabajos 2021/melisa/experimentos/05_experiment_beto/bert/ were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenc

In [22]:
from hyperparams_search import ut

df_train, df_dev = ut.train_dev_split(df,dev_size=0.1,random_state=293750254)
y_train, y_dev = df_train['review_rate'].values-1, df_dev['review_rate'].values-1

n = 1000
corpus_train = df_train['review_content'][:n].tolist()
y_train = y_train[:n]
corpus_dev = df_dev['review_content'][:n].tolist()
y_dev = y_dev[:n]

encoded_input = clf.tokenizer(corpus_train,
                            add_special_tokens=True,
                            padding=True,
                            truncation=True,
                            max_length=clf.max_len,
                            return_tensors='pt')

# print(corpus_train[0])
# print(' '.join(clf.tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])))
# print(encoded_input['attention_mask'])


clf.fit(corpus_train,y_train)

y_pred_train = clf.predict(corpus_train)
y_pred_dev = clf.predict(corpus_dev)

print('train accuracy: {:.2f}'.format((y_train == y_pred_train).mean()*100))
print('dev accuracy: {:.2f}'.format((y_dev == y_pred_dev).mean()*100))

Encoding inputs...
Training...
epoch 1/10. Loss: 1.0831
epoch 2/10. Loss: 0.5072
epoch 3/10. Loss: 0.5763
epoch 4/10. Loss: 1.2014
epoch 5/10. Loss: 0.3444
epoch 6/10. Loss: 0.1265
epoch 7/10. Loss: 0.4994
epoch 8/10. Loss: 0.0277
epoch 9/10. Loss: 0.9139
epoch 10/10. Loss: 0.1098
tensor([[-8.9666e-01, -1.3627e+00, -1.8729e+00, -1.2824e+00,  5.9539e+00],
        [-2.4201e+00, -1.5934e+00, -5.1024e-01,  5.2075e+00,  1.4027e-02],
        [-2.5058e+00, -1.7964e-01,  5.5233e+00, -6.8502e-01, -2.4999e+00],
        [-2.3479e+00, -2.1123e+00, -9.6281e-01,  4.2620e+00,  1.9186e+00],
        [-2.0446e+00, -8.4209e-01,  5.7524e+00, -1.2947e+00, -2.1673e+00],
        [ 5.4518e+00,  1.1773e+00, -1.6628e+00, -3.1601e+00, -2.1520e+00],
        [-2.2285e+00, -2.0315e+00, -1.2954e+00,  5.2044e+00,  1.5321e+00],
        [-2.9956e-01,  5.6669e+00, -1.0721e+00, -2.2317e+00, -2.1479e+00],
        [-1.1745e+00, -1.2004e+00, -1.8668e+00, -1.2376e+00,  5.9627e+00],
        [-1.9632e+00, -9.5131e-01, -9.5467e

In [20]:


for i in range(100):
    output = clf.model(input_ids=encoded_input['input_ids'][i].unsqueeze(0),
                attention_mask=encoded_input['attention_mask'][i].unsqueeze(0),
                token_type_ids=encoded_input['token_type_ids'][i].unsqueeze(0),
                output_hidden_states=False,
                output_attentions=False,
                return_dict=True)
    logprobs = output['logits']
    print(logprobs)

tensor([[-0.3903,  0.3744, -0.3385, -0.1665, -0.1023]],
       grad_fn=<AddmmBackward>)
tensor([[-0.4514,  0.3769, -0.0845,  0.1110,  0.0202]],
       grad_fn=<AddmmBackward>)
tensor([[-0.2777,  0.3027,  0.0871,  0.0106, -0.1499]],
       grad_fn=<AddmmBackward>)
tensor([[-0.4580,  0.3089, -0.1618, -0.1304, -0.2624]],
       grad_fn=<AddmmBackward>)
tensor([[-0.4634,  0.0581, -0.1334,  0.0711, -0.0923]],
       grad_fn=<AddmmBackward>)
tensor([[-0.3906,  0.0852,  0.0297,  0.1036, -0.3155]],
       grad_fn=<AddmmBackward>)
tensor([[-0.3914,  0.2275, -0.4019,  0.0524, -0.0581]],
       grad_fn=<AddmmBackward>)
tensor([[-0.2517,  0.3507,  0.0481, -0.2897, -0.0813]],
       grad_fn=<AddmmBackward>)
tensor([[-0.3049,  0.1107, -0.1574,  0.2140, -0.0615]],
       grad_fn=<AddmmBackward>)
tensor([[-0.2812,  0.2996, -0.3518,  0.1347, -0.1330]],
       grad_fn=<AddmmBackward>)
tensor([[-0.3880,  0.1788, -0.4373, -0.0575,  0.1008]],
       grad_fn=<AddmmBackward>)
tensor([[-0.5244,  0.0534, -0.01

In [ ]:
from sympy import Matrix
A = Matrix([[1,2,3]])
v, p = A.nullspace()
v1 = v.subs({p[0]:1, p[1]:0}) # Evalúo v con tau_0 = 1 y tau_1 = 0
v2 = v.subs({p[0]:0, p[1]:1}) # Evalúo v con tau_0 = 0 y tau_1 = 1